<a href="https://colab.research.google.com/github/gibranfp/CursoAprendizajeProfundo/blob/2025-1/notebooks/1a_neuronas_artificiales.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neuronas artificiales
---
La neurona artificial es un modelo simplificado de la neurona natural, la cual trata de imitar 3 aspectos principales:

1. La fuerza sináptica que pondera los impulsos recibidos
2. La acumulación de estos impulsos ponderados
3. La activación de la neurona que produce un impulso de respuesta a su salida.

La primera neurona artificial fue la llamada Unidad de Umbral Lineal (LTU por las siglas en inglés de _Linear Threshold Unit_), propuesta en 1943 por Warren McCulloch y Walter Pitts, la cual presupone que tanto los valores de los atributos de entrada como los valores de salida son binarios.

In [1]:
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn import datasets

## Unidad de Umbral Lineal
La operación que lleva a cabo una neurona articial está dada por la suma pesada evaluada con una función de activación $\phi$. Una de las primeras funciones de activación utilizadas fue la escalón unitario, definida como

$
\phi(x) = \begin{cases} 1, & \text{si } x \geq 0\\0, & \text{en caso contrario}\end{cases}
$

Esta se puede llevar a cabo con la siguiente función de Python:

In [2]:
def escalon(z):
  if z >= 0.0:
    return 1.0
  else:
    return 0.0

Por su parte, la suma pesada simplemente consiste en multiplicar cada entrada por su correspondiente peso y sumarle el sesgo. Esto lo podemos expresar como

$
z = w_1 \cdot x_1 + w_2 \cdot x_2 + \cdots + w_d \cdot x_d + b
$

En su forma vectorial

$
z = \mathbf{w}^T \mathbf{x} + b
$

Para realizar esto en Python, podemos usar la función `dot` de NumPY de la siguiente manera `z = np.dot(w.T, x) + b`. Así, la operación de la neurona completa sería:

In [3]:
def neurona(x, w, b):
  z = np.dot(w.T, x) + b
  return escalon(z)

### OR ($\lor$)
Esta neurona es capaz de aproximar la compuerta OR, cuya salida es 1 cuando al menos 1 de las 2 entradas es 1:


| $x_1$ | $x_2$ | $y$
| ------------- |:-------------:| -----:|
|0 |0 |0|
|0 |1 |1|
|1 |0 |1|
|1 |1 |1|

La neurona recibe 2 valores binarios como entrada y produce un valor binario como salida. Específicamente, la neurona calcularía

$
\hat{y} = \phi(w_1 \cdot x_1 + w_2 \cdot x_2 + b)
$

Para poder la compuerta OR es necesario encontrar los valores apropiados de $w_1$, $w_2$ y $b$. Una posible elección sería 10, 10 y -5 respectivamente. Verifiquemos estos valores:

In [4]:
X = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
w = np.array([10, 10])
b = -5

print('-----------------------------')
print('x_1 \tx_2 \ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
  y_hat = neurona(X[i, :].T, w, b)
  print('{0} \t{1}\t{2}'.format(X[i, 0], X[i, 1], y_hat))

-----------------------------
x_1 	x_2 	y_hat
-----------------------------
0.0 	0.0	0.0
0.0 	1.0	1.0
1.0 	0.0	1.0
1.0 	1.0	1.0


### AND ($\land$)
De forma similar, podemos aproximar la computerta AND:


| $x_1$ | $x_2$ | $y$
| ------------- |:-------------:| -----:|
|0 |0 |0|
|0 |1 |0|
|1 |0 |0|
|1 |1 |1|

Nuevamente, debemos encontrar los valores apropiados para los pesos y el sesgo. Probemos con $w_1 = 10$, $w_2 = 10$ y $b = -15$.

In [5]:
w = np.array([10, 10])
b = -15

print('-----------------------------')
print('x_1 \tx_2 \ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
  y_hat = neurona(X[i, :].T, w, b)
  print('{0} \t{1}\t{2}'.format(X[i, 0], X[i, 1], y_hat))

-----------------------------
x_1 	x_2 	y_hat
-----------------------------
0.0 	0.0	0.0
0.0 	1.0	0.0
1.0 	0.0	0.0
1.0 	1.0	1.0


### NAND
También podemos aproximar la computerta NAND:


| $x_1$ | $x_2$ | $y$
| ------------- |:-------------:| -----:|
|0 |0 |1|
|0 |1 |1|
|1 |0 |1|
|1 |1 |0|

En este caso, los valores para los pesos y el sesgo son $w_1 = -10$, $w_2 = -10$ y $b = 15$.

In [6]:
w = np.array([-10, -10])
b = 15

print('-----------------------------')
print('x_1 \tx_2 \ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
  y_hat = neurona(X[i, :].T, w, b)
  print('{0} \t{1}\t{2}'.format(X[i, 0], X[i, 1], y_hat))

-----------------------------
x_1 	x_2 	y_hat
-----------------------------
0.0 	0.0	1.0
0.0 	1.0	1.0
1.0 	0.0	1.0
1.0 	1.0	0.0


## Algoritmo Perceptrón
El Perceptron es un algoritmo para entrenar neuronas artificiales  propuesto por Frank Rosenblatt en 1958. El procedimiento general es el siguiente:

1. En $t=0$, inicializa los pesos $\mathbf{w}^{[0]}$ y el sesgo $b^{[0]}$ con ceros o números aleatorios pequeños
2. Para cada ejemplo $(\mathbf{x}^{(i)}, \mathbf{y}^{(i)})$ en el conjunto de entrenamiento:

    a. Calcula la salida:

    $$\hat{y}^{(i)} = \phi(\mathbf{w}^{[t]\top} \mathbf{x}^{(i)} + b^{[t]})$$

    b. Actualiza cada peso $w_j, j = 1, \ldots, d$ y el sesgo $b$:

\begin{align*}
w_j^{[t + 1]} = & w_j^{[t]} + (y^{(i)} - \hat{y}^{(i)})\cdot x^{(i)}_j\\
b^{[t + 1]} = & b^{[t]} + (y^{(i)} - \hat{y}^{(i)})
\end{align*}

3. Repite 2 hasta que se cumpla algún criterio de convergencia

In [7]:
def perceptron(X, y, n_epochs = 10):
  w = np.zeros(X.shape[1])
  b = 0
  for i in range(n_epochs):
    serr = 0.0
    for j in range(X.shape[0]):
      y_hat = neurona(X[j].T, w, b)
      error = y[j] - y_hat

      w += error * X[j].T
      b += error

      serr += np.abs(error)

    print("Epoch {0}: error = {1}".format(i, serr / float(X.shape[0])))

  return w, b

### Aprendiendo la compuerta OR
Probemos el algoritmo del perceptrón para aprender la computerta OR.

In [8]:
X = np.array([[0., 0.], [0., 1.], [1., 0.], [1., 1.]])
y_or = np.array([0., 1., 1., 1.])

w, b = perceptron(X, y_or)

print('\nw_1 = {0}, w_2 = {1}, b = {2}'.format(w[0], w[1], b))
print('-----------------------------')
print('x_1 \tx_2 \t y\ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
  y_hat = neurona(X[i].T, w, b)
  print('{0}\t{1}\t{2}\t{3}'.format(X[i, 0], X[i, 1], y_or[i], y_hat))

Epoch 0: error = 0.25
Epoch 1: error = 0.5
Epoch 2: error = 0.25
Epoch 3: error = 0.0
Epoch 4: error = 0.0
Epoch 5: error = 0.0
Epoch 6: error = 0.0
Epoch 7: error = 0.0
Epoch 8: error = 0.0
Epoch 9: error = 0.0

w_1 = 1.0, w_2 = 1.0, b = 0.0
-----------------------------
x_1 	x_2 	 y	y_hat
-----------------------------
0.0	0.0	0.0	0.0
0.0	1.0	1.0	1.0
1.0	0.0	1.0	1.0
1.0	1.0	1.0	1.0


### Aprendiendo la operación AND
Ahora veamos qué ocurre si en lugar de la OR aprendemos la compuerta AND

In [9]:
y_and = np.array([0., 0., 0., 1.])
w, b = perceptron(X, y_and)

print('\nw_1 = {0}, w_2 = {1}, b = {2}'.format(w[0], w[1], b))
print('-----------------------------')
print('x_1 \tx_2 \t y\ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
  y_hat = neurona(X[i].T, w, b)
  print('{0}\t{1}\t{2}\t{3}'.format(X[i, 0], X[i, 1], y_and[i], y_hat))

Epoch 0: error = 0.25
Epoch 1: error = 0.75
Epoch 2: error = 0.75
Epoch 3: error = 0.5
Epoch 4: error = 0.25
Epoch 5: error = 0.0
Epoch 6: error = 0.0
Epoch 7: error = 0.0
Epoch 8: error = 0.0
Epoch 9: error = 0.0

w_1 = 2.0, w_2 = 1.0, b = -2.0
-----------------------------
x_1 	x_2 	 y	y_hat
-----------------------------
0.0	0.0	0.0	0.0
0.0	1.0	0.0	0.0
1.0	0.0	0.0	0.0
1.0	1.0	1.0	1.0


## Aproximando funciones no lineales: XOR ($\oplus$)
Minsky y Papert mostraron que una neurona del tipo LTU no puede aproximar de forma precisa una función no lineal como la compuerta XOR ($\oplus$):


| $x_1$ | $x_2$ | $y$
| ------------- |:-------------:| -----:|
|0 |0 |0|
|0 |1 |1|
|1 |0 |1|
|1 |1 |0|

Sin embargo, es posible aproximarlas combinando múltiples neuronas LTU conectadas en red. Por ejemplo, es posible llevar a cabo la compuerta XOR con compuertas OR, AND y NAND:

$
	  x_1 \mathbin{\oplus} x_2 = (x_1 \lor x_2) \land \neg(x_1 \land x_2)
	$  

Esto lo llevamos a cabo con la siguiente función:

In [10]:
def multicapa(x, W1, b1, W2, b2):
  escv = np.vectorize(escalon)
  a = escv(np.dot(W1.T, x) + b1)
  return escv(np.dot(W2.T, a) + b2)

Encontrando los valores de pesos y sesgos adecuados, podemos usar esta función para aproximar la computerta XOR. Ya hemos encontrado los pesos y sesgos para las compuertas OR, AND y NAND, por lo que podemos usar estas neuronas con sus correspondientes pesos y sesgos. La red tendría 2 neuronas conectadas a las entradas que realizan las operaciones OR ($w_{11}^{\{1\}} = 10$, $w_{12}^{\{1\}} = 10$ y $b_1^{\{1\}} = -5$)  y NAND ($w_{21}^{\{1\}} = -10$, $w_{22}^{\{1\}} = -10$ y $b_2^{\{1\}} = 15$) respectivamente. La salida de estas 2 neuronas estarían conectadas a una tercera neurona que realiza la operacioón AND ($w_{11}^{\{2\}} = 10$, $w_{12}^{\{2\}} = 10$ y $b_1^{\{2\}} = -15$). En su forma matricial:

$$
\mathbf{W}^{\{1\}} = \left[\begin{matrix}
10 & -10\\
10 & -10
\end{matrix}\right]
$$

$$
\mathbf{b}^{\{1\}} = \left[\begin{matrix}
-5 \\
15
\end{matrix}\right]
$$

$$
\mathbf{W}^{\{2\}} = \left[\begin{matrix}
10\\
10
\end{matrix}\right]
$$

$$
\mathbf{b}^{\{2\}} = \left[\begin{matrix}
-15\\
\end{matrix}\right]
$$

In [11]:
y_xor = np.array([0., 1., 1., 0.])
W1 = np.array([[10, -10], [10, -10]])
b1 = np.array([-5, 15])

W2 = np.array([[10], [10]])
b2 = np.array([-15])

print('W_1 = [{0}{1}], b_1 = {2}'.format(W1[0, :], W1[1, :], b1))
print('W_2 = [{0}{1}], b_2 = {2}'.format(W2[0], W2[1], b2))
print('-----------------------------')
print('x_1 \tx_2 \ty\ty_hat')
print('-----------------------------')
for i in range(X.shape[0]):
  y_hat = multicapa(X[i].T, W1, b1, W2, b2)
  print('{0}\t{1}\t{2}\t{3}'.format(X[i, 0], X[i, 1], y_xor[i], y_hat[0]))

W_1 = [[ 10 -10][ 10 -10]], b_1 = [-5 15]
W_2 = [[10][10]], b_2 = [-15]
-----------------------------
x_1 	x_2 	y	y_hat
-----------------------------
0.0	0.0	0.0	0.0
0.0	1.0	1.0	1.0
1.0	0.0	1.0	1.0
1.0	1.0	0.0	0.0
